 ···················································


In [1]:
import openai
import os

In [18]:


# Load the API key from an environment variable or secret management service
openai.api_key = key; #os.getenv("OPENAI_API_KEY")


def chatgpt_query_to_text(prompt,
                          model="gpt-4",
                         #  engine="davinci",
                         # max_tokens=2000,
                          n=1,
                          stop=None,
                          temperature=0):
    # Call the OpenAI API to generate answers
    response = openai.Completion.create(
        model="gpt-4",
        # engine=engine,
        prompt=prompt,
        #max_tokens=max_tokens,
        n=n,
        stop=stop,
        temperature=temperature)
    print(response)
    response.choices[0].text

In [20]:
text2 = chatgpt_query_to_text(prompt)

InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?

In [6]:
text = chatgpt_query_to_text("Write a brief summary about the RBL2 gene")

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " and the protein it encodes.\n\nThe RBL2 gene is located on chromosome 1. It is a member of the RBL gene family. The RBL2 gene encodes a protein called RBL2 protein. The RBL2 protein is a member of the RBL family of proteins. The RBL2 protein is a protein kinase. It is a serine/threonine protein kinase. It is a member of the RBL family of serine/threonine protein kinases. The RBL2 protein is a protein kinase that is involved in the regulation of the cell cycle. It is involved in the regulation of the G1/S transition. It is involved in the regulation of the G1/S transition by phosphorylating the retinoblastoma protein. The RBL2 protein is involved in the regulation of the G1/S transition by phosphorylating the retinoblastoma protein. The RBL2 protein is involved in the regulation of the G1/S transition by phosphorylating the retinoblastoma protein. The RBL2 protein is involve

In [ ]:
openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},

    ]
)

In [ ]:
response = openai.ChatCompletion.create(
    #model="gpt-3.5-turbo",
    model="gpt-4",
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]
)

print(response)

print(response['choices'][0]['message']['content'])

In [ ]:
import os
import json
import openai
from datetime import datetime

# Set the OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

LOG_FILE = "log.json"
DOLLAR_LIMIT = 1.0  # Set your dollar limit here

def load_log():
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r") as f:
            return json.load(f)
    else:
        return {"tokens_used": 0, "dollars_spent": 0.0}

def save_log(log_data):
    with open(LOG_FILE, "w") as f:
        json.dump(log_data, f, indent=4)

def estimate_cost(tokens, rate_per_token):
    return tokens * rate_per_token

def openai_chat(prompt, model, max_tokens, rate_per_token):
    log_data = load_log()
    tokens_estimate = len(prompt) + max_tokens

    if estimate_cost(log_data["tokens_used"] + tokens_estimate, rate_per_token) > DOLLAR_LIMIT:
        print("The API call is estimated to exceed the dollar limit. Aborting.")
        return

    try:
        response = openai.Completion.create(
            engine=model,
            prompt=prompt,
            max_tokens=max_tokens,
            n=1,
            stop=None,
            temperature=1,
        )

        tokens_used = response["choices"][0]["usage"]["total_tokens"]

        # Update and save the log
        log_data["tokens_used"] += tokens_used
        log_data["dollars_spent"] = estimate_cost(log_data["tokens_used"], rate_per_token)
        save_log(log_data)

        return response["choices"][0]["text"]
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
if __name__ == "__main__":
    prompt = "What are some tips for productivity?"
    model = "text-davinci-002"
    max_tokens = 50
    rate_per_token = 0.0005  # Set your rate per token (in dollars)

    response_text = openai_chat(prompt, model, max_tokens, rate_per_token)
    if response_text:
        print(response_text)
